In [1]:
import numpy as np
from scipy.optimize import linprog
from bac import solve_mip

# Problem parameters
num_facilities = 3
num_customers = 5
num_scenarios = 100
num_periods = 4  # Number of time periods

# Fixed costs for opening facilities
fixed_costs = np.array([100000, 120000, 90000])

# Generate variable transportation costs
np.random.seed(42)
base_trans_costs = np.array([
    [10, 15, 20, 25, 30],
    [12, 14, 18, 22, 26],
    [8, 16, 24, 32, 40]
])
trans_costs_scenarios = np.random.normal(base_trans_costs, 2, (num_scenarios, num_periods, num_facilities, num_customers))
trans_costs_scenarios = np.clip(trans_costs_scenarios, 0, None)  # Ensure non-negative costs

# Facility capacities
capacities = np.array([5000, 6000, 4500])

# Generate random demand scenarios
demand_mean = 1000
demand_std = 200
demand_scenarios = np.random.normal(demand_mean, demand_std, (num_scenarios, num_periods, num_customers))
demand_scenarios = np.clip(demand_scenarios, 0, None)  # Ensure non-negative demand

# Penalty parameter for the algorithm
rho = 1.0

# Number of iterations
max_iterations = 100

# Convergence tolerance
tolerance = 1e-4

# Initialize decision variables and dual variables
x = np.zeros((num_scenarios, num_periods, num_facilities, num_customers))
y = np.zeros((num_scenarios, num_facilities))  # Binary decision for opening facilities
lambda_x = np.zeros((num_scenarios, num_periods, num_facilities, num_customers))
lambda_y = np.zeros((num_scenarios, num_facilities))

def solve_subproblem(demands, trans_costs, fixed_costs, capacities, lambda_x, lambda_y, x_avg, y_avg, rho, s):
    num_facilities, num_customers = trans_costs[0].shape
    num_periods = demands.shape[0]
    num_vars = num_facilities + num_facilities * num_customers * num_periods

    # Objective function coefficients
    c = np.zeros(num_vars)
    c[:num_facilities] = fixed_costs
    c[num_facilities:] = trans_costs.flatten()

    # Add augmented Lagrangian terms
    for i in range(num_facilities):
        c[i] += lambda_y[i] * 1 + 0.5 * rho * (1 - y_avg[i])**2
    for t in range(num_periods):
        for i in range(num_facilities):
            for j in range(num_customers):
                idx = num_facilities + (t * num_facilities * num_customers) + (i * num_customers) + j
                c[idx] += lambda_x[t, i, j] * 1 + 0.5 * rho * (1 - x_avg[t, i, j])**2

    # Constraints
    A_ub = []
    b_ub = []

    # Capacity constraints for each period
    for t in range(num_periods):
        for i in range(num_facilities):
            row = np.zeros(num_vars)
            row[i] = -capacities[i]
            start = num_facilities + (t * num_facilities * num_customers) + (i * num_customers)
            end = start + num_customers
            row[start:end] = 1
            A_ub.append(row)
            b_ub.append(0)

    # Demand satisfaction constraints for each period
    A_eq = []
    b_eq = []
    for t in range(num_periods):
        for j in range(num_customers):
            row = np.zeros(num_vars)
            for i in range(num_facilities):
                idx = num_facilities + (t * num_facilities * num_customers) + (i * num_customers) + j
                row[idx] = 1
            A_eq.append(row)
            b_eq.append(demands[t, j])

    # Convert to numpy arrays
    A_ub = np.array(A_ub)
    b_ub = np.array(b_ub)
    A_eq = np.array(A_eq)
    b_eq = np.array(b_eq)

    # Bounds
    bounds = [(0, 1)] * num_facilities + [(0, None)] * (num_vars - num_facilities)

    int_vars = np.zeros(len(c), dtype=int)
    int_vars[:num_facilities] = 1
    int_vars = int_vars.tolist()

    # Solve the problem
    result = solve_mip(c, A_ub, b_ub, A_eq, b_eq, bounds, int_vars, minimize=True)

    # Extract results
    y = result[0][:num_facilities]
    x = result[0][num_facilities:].reshape(num_periods, num_facilities, num_customers)
    obj = result[1]

    return x, y, obj

# Progressive Hedging Algorithm
for iteration in range(max_iterations):
    x_old = x.copy()
    y_old = y.copy()
    
    # Compute averages
    x_avg = np.mean(x, axis=0)
    y_avg = np.mean(y, axis=0)
    
    # Solve subproblems for each scenario
    obj_list = []
    for s in range(num_scenarios):
        x[s], y[s], obj = solve_subproblem(demand_scenarios[s], trans_costs_scenarios[s], fixed_costs, capacities, 
                                           lambda_x[s], lambda_y[s], x_avg, y_avg, rho, s)
        obj_list.append(obj)
    
    # Update dual variables
    for s in range(num_scenarios):
        lambda_x[s] += rho * (x[s] - x_avg)
        lambda_y[s] += rho * (y[s] - y_avg)
    
    # Check for convergence
    if np.max(np.abs(x - x_old)) < tolerance and np.max(np.abs(y - y_old)) < tolerance:
        print(f"Converged in {iteration + 1} iterations")
        break

    print(f"Iteration {iteration}, Avg Obj: {np.mean(obj_list)}")

# Display final decision variables
print("Facility opening decisions (y):")
print(np.round(y_avg))
print("Average transportation decisions (x) over all periods:")
print(np.round(np.mean(x_avg, axis=0)))

Iteration 0, Avg Obj: 820985.0771903074
Iteration 1, Avg Obj: 7262248781.451346
Iteration 2, Avg Obj: 9869296130.847322
Iteration 3, Avg Obj: 9874114129.160263
Converged in 5 iterations
Facility opening decisions (y):
[1. 1. 1.]
Average transportation decisions (x) over all periods:
[[   0.    7.    0.    2.    7.]
 [1011.    2.    0.    2.    0.]
 [   0.  996.  990.  990.  989.]]
